Imports

In [90]:
import cv2
import numpy as np
import os

In [91]:
show_steps = False

Constants

In [92]:
# Intrinsic Matrix
K = np.array([[2564.3186869, 0, 0], [0, 2569.70273111, 0], [0, 0, 1]])

# Circle Radius
R = 10.0

Functions

In [93]:
def find_most_circular_contour(contours):

    best_circularity = 0
    best_contour = None
    best_index = -1
    
    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        
        if perimeter > 0 and area > 500:  
            circularity = 4 * np.pi * area / (perimeter * perimeter)
            
            if circularity > best_circularity:
                best_circularity = circularity
                best_contour = contour
                best_index = i
    
    return best_contour, best_circularity, best_index

In [94]:
def calculate_3d_coordinates(contours, centers, camera_matrix, circle_radius):

    coordinates_3d = []
    reference_depth = None
    
    # Extract camera parameters
    fx = camera_matrix[0, 0]
    fy = camera_matrix[1, 1]
    cx = camera_matrix[0, 2]
    cy = camera_matrix[1, 2]
    
    # Find the most circular contour
    circle_contour, circularity, circle_index = find_most_circular_contour(contours)
    
    if circle_contour is not None and circularity > 0.8:

        # Calculate depth from the most circular object
        area = cv2.contourArea(circle_contour)
        radius_pixels = np.sqrt(area / np.pi)
        
        # Calculate reference depth
        reference_depth = (fx * circle_radius) / radius_pixels
            
    else:

        # Default depth
        reference_depth = 100.0 
    
    # Calculate 3D coordinates for all shapes using reference depth
    for i, (contour, center) in enumerate(zip(contours, centers)):

        # Image coordinates
        u, v = center  
        
        # Calculate 3D coordinates
        Z = reference_depth
        X = (u - cx) * Z / fx
        Y = (v - cy) * Z / fy
        coordinates_3d.append((X, Y, Z))
    
    return coordinates_3d, circle_index

In [95]:
def draw_3d_annotations(frame, contours, centers, coordinates_3d, circle_index=-1):
    
    result_frame = frame.copy()
    
    # Draw contours
    cv2.drawContours(result_frame, contours, -1, (0, 255, 0), 2)
    
    for i, (contour, center, coord_3d) in enumerate(zip(contours, centers, coordinates_3d)):
        cx, cy = center
        X, Y, Z = coord_3d
        
        # Get bounding box for text placement
        x, y, w, h = cv2.boundingRect(contour)
        
        # Draw center
        cv2.circle(result_frame, center, 5, (255, 255, 255), -1)

        # Place 3D coordinates
        text_3d = f"({X:.1f},{Y:.1f},{Z:.1f})"
        cv2.putText(result_frame, text_3d, (cx - 80, y + h + 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    return result_frame

In [96]:
def edge_consistency_mask(gray):
    
    # Find edges
    edges = cv2.Canny(gray, 50, 150)
    
    # Dilate edges to create boundaries
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    edge_dilated = cv2.dilate(edges, kernel, iterations=1)
    
    # Invert 
    consistent_mask = cv2.bitwise_not(edge_dilated)
    
    return consistent_mask

In [97]:
def remove_noise_and_erode(noise_mask):

    # Removes noise
    noise_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    no_noise_mask = cv2.morphologyEx(noise_mask, cv2.MORPH_OPEN, noise_kernel, iterations=1)

    # Erode to separate shapes connected to random blotches
    erosion_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    eroded_mask = cv2.erode(no_noise_mask, erosion_kernel, iterations=2)

    return eroded_mask

In [98]:
def smooth_mask_and_fill_gaps(unsmoothed_mask):
    
    smoothing_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))

    # Fill the gaps and smooth the edges
    filled_gaps_mask = cv2.morphologyEx(unsmoothed_mask, cv2.MORPH_CLOSE, smoothing_kernel)

    # Smooth more with blur
    smoothed_mask = cv2.medianBlur(filled_gaps_mask, 15)

    return smoothed_mask

In [99]:
def heavy_erode(non_eroded_mask):
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))

    # Remove small noise while keeping size of surviving shapes
    opened_mask = cv2.morphologyEx(non_eroded_mask, cv2.MORPH_OPEN, kernel, iterations=3)

    # Heavy erosion to separate shapes
    separated_mask = cv2.erode(opened_mask, kernel, iterations=7)

    return separated_mask

In [100]:
def filter_countours_compactness(old_mask, contours, min_area=1700, max_compactness=55, min_perimeter=0):

    # Create empty mask from old mask shape
    new_mask = np.zeros_like(old_mask)

    for contour in contours:

        # Get area/perimeter for each contour
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        
        # Filter by area and compactness of shape
        if area > min_area and perimeter > min_perimeter and (perimeter * perimeter) / area < max_compactness:
            # Draw passing contours on new mask
            cv2.fillPoly(new_mask, [contour], 255)
    
    return new_mask

In [101]:
def dilate_to_undo_erosion(clean_mask):

    # Undo erosion that filled gaps
    erosion_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))
    final_mask = cv2.dilate(clean_mask, erosion_kernel, iterations=2)

    # Undo heavy erosion that separated shapes
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
    final_mask = cv2.dilate(final_mask, kernel, iterations=7)

    return final_mask

In [102]:
def get_rigid_contours(contours, epsilon_size=0.01):

    rigid_contours = []

    for contour in contours:
        
        # Approximate contour with fewer points
        epsilon = epsilon_size * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        rigid_contours.append(approx)
    
    return rigid_contours

In [103]:
def get_hull_contours(contours):

    hull_contours = []
    
    for contour in contours:
        
        # Get convex hull of contour
        hull = cv2.convexHull(contour)
        hull_contours.append(hull)
    
    return hull_contours

In [104]:
def filter_noise_contours(contours, min_area=1000):

    # Remove noise
    filtered_contours = []

    for contour in contours:
        
        area = cv2.contourArea(contour)

        # How much noise to filter
        if area > 1000:
            filtered_contours.append(contour)
    
    return filtered_contours

In [105]:
def get_centers(contours):

    # Calculate centers of contours
    centers = []

    for contour in contours:

        # Calculate moments
        M = cv2.moments(contour)
        
        if M["m00"] != 0: 

            # x coord
            cx = int(M["m10"] / M["m00"])

            # y coord
            cy = int(M["m01"] / M["m00"])

            centers.append((cx, cy))
    
    return centers

File Names/Dirs

In [106]:
file_dir = "data"
file_name = "PennAir 2024 App Dynamic.mp4"
file = os.path.join(file_dir, file_name)

In [107]:
output_dir = os.path.join("output", "consistency")
output_name = f"annotated_3D_{file_name}"
output = os.path.join(output_dir, output_name)

Load Video

In [108]:
vid = cv2.VideoCapture(file)

In [109]:
# Video properties
fps = vid.get(cv2.CAP_PROP_FPS)
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Video writer to save output video
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out_vid = cv2.VideoWriter(output, fourcc, fps, (width, height))

Update K

In [110]:
# Update camera matrix principal point if not provided
K_updated = K.copy()
if K_updated[0, 2] == 0:
    K_updated[0, 2] = width / 2
if K_updated[1, 2] == 0:
    K_updated[1, 2] = height / 2

In [111]:
# Loop through video frames
while True:
    ret, frame = vid.read()

    # Break if no frame is returned
    if not ret:
        break
    
    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Consistency detection to create mask
    consistency_mask = edge_consistency_mask(gray)

    # Remove noise and erode
    cleaned_mask = remove_noise_and_erode(consistency_mask)

    # Smooth
    smooth_mask = smooth_mask_and_fill_gaps(cleaned_mask)

    # Erode
    eroded_mask = heavy_erode(smooth_mask)
    
    # Find contours
    contours, _ = cv2.findContours(eroded_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter contours by compactness
    clean_mask = filter_countours_compactness(eroded_mask, contours)

    # Undo erosion
    final_mask = dilate_to_undo_erosion(clean_mask)

    # Find/Draw Outlines and Centers
    contours, _ = cv2.findContours(final_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Make contours more rigid
    rigid_contours = get_rigid_contours(contours)

    # Make contours convex
    hull_contours = get_hull_contours(rigid_contours)

    filtered_contours = filter_noise_contours(hull_contours)
    
    centers = get_centers(filtered_contours)

    # Copy original frame
    result_frame = frame.copy()

    # Calculate 3D coordinates and get index in list of contours of most circular object
    coordinates_3d, circle_index = calculate_3d_coordinates(filtered_contours, centers, K_updated, R)
    
    # Draw results with 3D annotations
    result_frame = draw_3d_annotations(result_frame, filtered_contours, centers, coordinates_3d, circle_index)
    
    # Write output
    out_vid.write(result_frame)
    
    if show_steps:
        # Display results
        cv2.imshow('Video Results', result_frame)
        if cv2.waitKey(1) & 0xFF == 13:
            break

vid.release()
out_vid.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

KeyboardInterrupt: 